In [1]:
# !pip install -qqq -U git+https://github.com/huggingface/peft.git
# !pip install -qqq -U git+https://github.com/huggingface/accelerate.git
# !pip install -qqq bitsandbytes
# !pip install scipy
# !pip install google-cloud-vision
# !pip install --upgrade google-auth
# !pip install datasets
# !pip install evaluate
# !pip install transformers -U
# !pip install ultralytics

In [2]:
# import zipfile
# from tqdm import tqdm

# # Open the ZIP file
# with zipfile.ZipFile('/root/NA0219_production_batches/NA0219_IIMI0263_4.zip', 'r') as zip_ref:
#     # Get the list of files in the ZIP archive
#     file_list = zip_ref.namelist()
    
#     # Initialize tqdm progress bar
#     with tqdm(total=len(file_list), desc="Extracting files") as pbar:
#         # Extract each file
#         for file in file_list:
#             zip_ref.extract(file, '/root/NA0219_production_batches/NA0219_IIMI0263_4')
#             # Update the progress bar
#             pbar.update(1)

In [ ]:
import os
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from collections import OrderedDict
from google.cloud import vision_v1
from google.cloud.vision_v1 import types
import json
import re
import logging
import time
import zipfile
import csv
import cv2
import shutil
from pprint import pprint
from datasets import load_dataset
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from typing import List
from datetime import datetime
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from accelerate import Accelerator
from accelerate.utils import gather_object
from itertools import groupby
from operator import itemgetter
from ultralytics import YOLO
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model = YOLO('/root/NA0219/runs/detect/train2/weights/last.pt')

In [4]:
# #  Loading config file
# def load_config(file_path):
#     with open(file_path, 'r') as file:
#         config = json.load(file)
#     return config

# json_file_path = '/root/NA0219/config_219.json'

# configurations = load_config(json_file_path)

# compressed_file_path = configurations['zip_file_path']
# folder_to_process_path = configurations['directory_path']
# output_txt_path = os.path.splitext(os.path.basename(folder_to_process_path))[0]

# print("Compressed File Path:", compressed_file_path)
# print("Folder to process:", folder_to_process_path)

# # Deleting the extra file after unzipping
# with zipfile.ZipFile(compressed_file_path, 'r') as zip_ref: zip_ref.extractall(folder_to_process_path)

# # If there are subfolders, move their contents to the base folder and delete the subfolders
# subfolders = [f for f in os.listdir(folder_to_process_path) if os.path.isdir(os.path.join(folder_to_process_path, f))]
# for subfolder in subfolders:
#     subfolder_path = os.path.join(folder_to_process_path, subfolder)
#     for item in os.listdir(subfolder_path):
#         item_path = os.path.join(subfolder_path, item)
#         new_item_path = os.path.join(folder_to_process_path, item)
#         shutil.move(item_path, new_item_path)
#     # Remove the now-empty subfolder
#     os.rmdir(subfolder_path)

# # # move folder
# # #shutil.move(folder_to_process_path, "/root/")

In [5]:
base_model_id = "HuggingFaceH4/zephyr-7b-alpha"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=4096,
    padding=True,
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token
ft_model = PeftModel.from_pretrained(base_model, "/root/NA0219/zephyr_7B_alpha-finetune-22422_NA0219_9_sep_2024/checkpoint-22000")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
def google_cloud_vision_response(img):
    try:
        success, encoded_image = cv2.imencode('.jpg', img)
        if not success:
            raise ValueError("Failed to encode image")

        content = encoded_image.tobytes()
        imgocr = vision_v1.Image(content=content)
        image_context = vision_v1.ImageContext(language_hints=['nl', 'de'])

        response = vision_v1.ImageAnnotatorClient().document_text_detection(image=imgocr, image_context=image_context)
        if not response:
            raise ValueError("No response from Google Cloud Vision")

        return response

    except Exception as e:
        print("Error in Google Cloud Vision API:", e)
        return None

bounding_boxes_info = []

def detect_crop_and_order_objects(img_path):
    # Step 1: Detect objects and get bounding boxes
    img = cv2.imread(img_path)
    detect_result = model(img)
    bboxes = []
    
    if detect_result[0].boxes.data.shape[0] > 0:
        for box in detect_result[0].boxes.data.tolist():
            xmin, ymin, xmax, ymax, conf, cls = box
            bboxes.append([img_path, cls, int(xmin), int(ymin), int(xmax), int(ymax), conf])
    
    # Step 2: Order the detections sequentially
    img_height, img_width = img.shape[:2]
    
    # Find the midpoint to split left and right sides
    midpoint = img_width / 2
    
    # Separate bounding boxes into left and right sides based on x-coordinates
    left_records = [bbox for bbox in bboxes if bbox[2] + bbox[4] / 2 < midpoint]  # center of bbox for left side
    right_records = [bbox for bbox in bboxes if bbox[2] + bbox[4] / 2 >= midpoint]  # center of bbox for right side
    
    # Sort left side (top-to-bottom based on ymin)
    left_records_sorted = sorted(left_records, key=lambda x: x[3])  # ymin for sorting
    
    # Sort right side (top-to-bottom based on ymin)
    right_records_sorted = sorted(right_records, key=lambda x: x[3])  # ymin for sorting
    
    # Merge both sequences: left side first, then right side
    ordered_bboxes = left_records_sorted + right_records_sorted
    
    # Extract coordinates
    coordinates = [(bbox[2], bbox[3], bbox[4], bbox[5]) for bbox in ordered_bboxes]  # xmin, ymin, xmax, ymax
    
    # Update global list with image name and ordered coordinates
    bounding_boxes_info.append({
        'Image': img_path,
        'coordinates': coordinates
    })
    
    return ordered_bboxes

def get_response_inside_roi(response, bboxes):
    words = []
    confidences = []
    bounding_boxes = []
    image_names = []  # List to store image names
    for bbox in bboxes:
        img_path, cls, xmin, ymin, xmax, ymax, conf = bbox
        image_names.append(os.path.basename(img_path))  # Extract image name from the image path

        for page in response.full_text_annotation.pages:
            for block in page.blocks:
                for paragraph in block.paragraphs:
                    for word in paragraph.words:
                        word_text = ''.join([symbol.text for symbol in word.symbols])
                        x1 = word.bounding_box.vertices[0].x
                        y1 = word.bounding_box.vertices[0].y
                        x2 = word.bounding_box.vertices[2].x
                        y2 = word.bounding_box.vertices[2].y

                        word_text = ''.join([symbol.text for symbol in word.symbols])
                        confidences.append((word_text, word.confidence))
                        vertices = [(v.x, v.y) for v in word.bounding_box.vertices]
                        bounding_boxes.append((word_text, vertices))

                        if xmin <= x1 <= xmax and ymin <= y1 <= ymax and xmin <= x2 <= xmax and ymin <= y2 <= ymax:
                            words.append(word_text)

    if words:
        combined_words = ' '.join(words)
        return image_names, combined_words, confidences, bounding_boxes

    return None

ocrs = []

def process_image(img_path, bboxes, ft_model, tokenizer):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read the image {img_path}.")
        # return output_format

    response = google_cloud_vision_response(img)
    processed_image_names = {}

    if response is not None:
        for bbox in bboxes:
            response_inside_roi = get_response_inside_roi(response, [bbox])
            if response_inside_roi:
                image_name, ocr_data, confidences, bounding_boxes = response_inside_roi

                # Handle duplicate image names
                base_image_name = image_name[0].rsplit('.', 1)[0]
                part_number = processed_image_names.get(base_image_name, 0) + 1
                unique_image_name = f"{base_image_name}_part{part_number}.{image_name[0].rsplit('.', 1)[1]}"
                processed_image_names[base_image_name] = part_number

                ocr_texts = [ocr_data]
                
                ocrs.append({
                    'Image': unique_image_name,
                    'OCR': ocr_data,
                    'Confidence Scores': confidences,
                    'Bounding Boxes': bounding_boxes
                })
    else:
        print(f"No response from Google Cloud Vision for {img_path}")

    return ocrs

def process_image_folder(input_folder, max_workers):
    images = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    output_format = []

    # img_bboxes = {img: detect_and_crop_objects(os.path.join(input_folder, img)) for img in images}
    img_bboxes = {img: detect_crop_and_order_objects(os.path.join(input_folder, img)) for img in images}

    with ThreadPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(images), desc='Processing Images') as pbar:
        futures = []
        for img_name in images:
            img_path = os.path.join(input_folder, img_name)
            future = executor.submit(process_image, img_path, img_bboxes[img_name], ft_model, tokenizer)
            future.add_done_callback(lambda p: pbar.update())
            futures.append(future)

        for future in futures:
            output_format.extend(future.result())

    return output_format

def get_image_ocr_list(dict_list):
    image_ocr_list = []
    for d in dict_list:
        if 'OCR' in d and 'Image' in d:
            image_ocr_dict = {d['Image']: d['OCR']}
            image_ocr_list.append(image_ocr_dict)
        elif 'Image' in d:
            image_ocr_dict = {d['Image']: None}
            image_ocr_list.append(image_ocr_dict)
    return image_ocr_list


all_outputs = []

def process_ocr_data(ocr_data, ft_model, tokenizer, batch_size=20, n_batches_before_error=500):
    batches_processed = 0

    try:
        ft_model.eval()

        print(f"Total OCR entries: {len(ocr_data)}")

        for i in tqdm(range(0, len(ocr_data), batch_size), desc="Processing batches", unit="batch"):
            while True:
                try:
                    batch_data = ocr_data[i:i + batch_size]

                    print(f"\nProcessing batch from index {i} to {i + batch_size}")

                    batch_images = [list(d.keys())[0] for d in batch_data]
                    batch_texts = [list(d.values())[0] for d in batch_data]

                    eval_prompts = [
                        f"""Extract specific data from the provided OCR text of a marriage record. Identify and return the following details in a structured JSON format: Application number, date of marriage ( month,day, year), place of marriage, birth details (month,day, year, place) of both spouses, their county, gender, given names, residence, state, and surnames. Only show the extracted information. Don't show the OCR Text again. Also don't show the pattern which you are asked to follow. Concentrate on the data asked to extract . The output must strictly adhere to the given details, starting with an opening curly brace '{{' and ending with a closing curly brace '}}'. Ensure that each key-value pair contains only accurate and relevant information from the OCR text. Avoid including any extraneous or irrelevant data that does not correspond directly to the specified keys. The structure of the output is as follows:

Event_Day_orig: The original day of the event.
Event_Month_orig: The original month of the event.
Event_Year_orig: The original year of the event.
Given_Name_orig: The given name of the person.
Surname_orig: The surname of the person.
Sex_orig: The gender of the person.
Age_orig: The age of the person.
Birth_Year_orig: The original year of birth of the person.
Death_Year_orig: The original year of death of the person.
Father_Given_Name_orig: The given name of the person's father.
Father_Surname_orig: The surname of the person's father.
Mother_Given_Name_orig: The given name of the person's mother.
Mother_Surname_orig: The surname of the person's mother.
Spouse_Given_Name_orig: The given name of the spouse.
Spouse_Surname_orig: The surname of the spouse.
Spouse_Age_orig: The age of the spouse.
Spouse_Birth_Year_orig: The original year of birth of the spouse.
Spouse_Father_Given_Name_orig: The given name of the spouse's father.
Spouse_Father_Surname_orig: The surname of the spouse's father.
Spouse_Mother_Given_Name_orig: The given name of the spouse's mother.
Spouse_Mother_Surname_orig: The surname of the spouse's mother.

### OCR Text:
{text}
""" for text in batch_texts]

                    batch_inputs = tokenizer(eval_prompts, padding=True, return_tensors="pt").to(ft_model.device)

                    with torch.no_grad():
                        batch_outputs = ft_model.generate(**batch_inputs, max_new_tokens=1000)
                        decoded_outputs = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)

                    batch_results = [{'Image': img, 'Model_Output': output} for img, output in zip(batch_images, decoded_outputs)]
                    all_outputs.extend(batch_results)

                    batches_processed += 1

                    if batches_processed >= n_batches_before_error:
                        raise torch.cuda.OutOfMemoryError("Intentionally triggered CUDA out of memory error after 500 batches.")

                    break

                except torch.cuda.OutOfMemoryError:
                    print("Threshold reached. Batch terminated")
                    raise

    except KeyboardInterrupt:
        print("Kernel interrupted. Saving current outputs and exiting.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return all_outputs

In [7]:
split_info =[]
def split_bbox_info(bbox_info):
    for entry in bbox_info:
        # Extract the base name of the image without the extension
        img_base_name = os.path.splitext(os.path.basename(entry['Image']))[0]
        
        # Loop through each set of coordinates and create a new entry
        for i, coords in enumerate(entry['coordinates'], start=1):
            split_info.append({
                'Image': f'{img_base_name}_part{i}.jpg',
                'coordinates': coords
            })
    
    return split_info
def is_within_coordinates(bbox, coordinates):
    x_min, y_min, x_max, y_max = coordinates
    x1, y1 = bbox[0]
    x2, y2 = bbox[1]
    x3, y3 = bbox[2]
    x4, y4 = bbox[3]
    return (x_min <= x1 <= x_max and y_min <= y1 <= y_max and
            x_min <= x2 <= x_max and y_min <= y2 <= y_max and
            x_min <= x3 <= x_max and y_min <= y3 <= y_max and
            x_min <= x4 <= x_max and y_min <= y4 <= y_max)

def extract_filtered_info_v2(data_list, split_info):
    filtered_ocrs = []

    # Create a dictionary to map images to their coordinates
    coordinates_map = {entry['Image']: entry['coordinates'] for entry in split_info}

    for data in data_list:
        if not isinstance(data, dict):
            raise TypeError("Each item in the input data must be a dictionary")

        image_name = data.get('Image', '')
        if image_name not in coordinates_map:
            print(f"Image '{image_name}' not found in split_info")
            continue
        
        ocr_text = data.get('OCR', '')
        confidence_scores_list = data.get('Confidence Scores', [])
        bounding_boxes_list = data.get('Bounding Boxes', [])

        confidence_dict = {word: score for word, score in confidence_scores_list}
        bounding_boxes_dict = {word: box for word, box in bounding_boxes_list}

        filtered_confidence_scores = []
        filtered_bounding_boxes = []
        filtered_ocr = []

        coords = coordinates_map[image_name]

        for word in ocr_text.split():
            if word in confidence_dict and word in bounding_boxes_dict:
                bbox = bounding_boxes_dict[word]

                if is_within_coordinates(bbox, coords):
                    filtered_confidence_scores.append((word, confidence_dict[word]))
                    filtered_bounding_boxes.append((word, bbox))
                    filtered_ocr.append(word)

        filtered_data = {
            'Image': image_name,
            'Coordinates': coords,
            'OCR': ' '.join(filtered_ocr),
            'Confidence Scores': filtered_confidence_scores,
            'Bounding Boxes': filtered_bounding_boxes
        }
        filtered_ocrs.append(filtered_data)

    return filtered_ocrs

In [8]:
def merge_lists(l1, l2):
    final_results = []
    
    for dict1 in l1:
        for dict2 in l2:
            if dict1['Image'] == dict2['Image']:
                merged_dict = dict2.copy()
                merged_dict.update(dict1)
                final_results.append(merged_dict)
                break
    
    return final_results

In [9]:
def extract_json_outputs_parallel(final_results, max_workers=8):
    """
    Parallel function to extract and modify JSON outputs from the results.
    :param final_results: List of dictionaries, each being an output from the previous function.
    :param max_workers: Maximum number of threads to use for parallel processing.
    :return: List of modified JSON outputs.
    """
    def process_item(item):
        # Accessing the string within the list for Model Output
        # inference_text = item['Model_output'][0]
        inference_text = item['Model_Output']
        
        pattern_1 = r"""
        Event_Day_orig:\s*(?P<Event_Day>\d*)\n
        Event_Month_orig:\s*(?P<Event_Month>[A-Za-z]*)\n
        Event_Year_orig:\s*(?P<Event_Year>\d*)\n
        Given_Name_orig:\s*(?P<Given_Name>[\w\s]*)\n
        Surname_orig:\s*(?P<Surname>[\w\s]*)\n
        Sex_orig:\s*(?P<Sex>\w*)\n
        Age_orig:\s*(?P<Age>\d*)\n
        Birth_Year_orig:\s*(?P<Birth_Year>\d*)\n
        Death_Year_orig:\s*(?P<Death_Year>\d*)\n
        Father_Given_Name_orig:\s*(?P<Father_Given_Name>[\w\s]*)\n
        Father_Surname_orig:\s*(?P<Father_Surname>[\w\s]*)\n
        Mother_Given_Name_orig:\s*(?P<Mother_Given_Name>[\w\s]*)\n
        Mother_Surname_orig:\s*(?P<Mother_Surname>[\w\s]*)\n
        Spouse_Given_Name_orig:\s*(?P<Spouse_Given_Name>[\w\s]*)\n
        Spouse_Surname_orig:\s*(?P<Spouse_Surname>[\w\s]*)\n
        Spouse_Age_orig:\s*(?P<Spouse_Age>\d*)\n
        Spouse_Birth_Year_orig:\s*(?P<Spouse_Birth_Year>\d*)\n
        Spouse_Father_Given_Name_orig:\s*(?P<Spouse_Father_Given_Name>[\w\s]*)\n
        Spouse_Father_Surname_orig:\s*(?P<Spouse_Father_Surname>[\w\s]*)\n
        Spouse_Mother_Given_Name_orig:\s*(?P<Spouse_Mother_Given_Name>[\w\s]*)\n
        Spouse_Mother_Surname_orig:\s*(?P<Spouse_Mother_Surname>[\w\s]*)\n
        """
        
        pattern_2 = r'### JSON Output:[\s\S]*?({[\s\S]*?})'
        
        # Compile the patterns with the verbose flag for readability
        regex_1 = re.compile(pattern_1, re.VERBOSE)
        regex_2 = re.compile(pattern_2)
        
        # Match both patterns against the inference text
        match_1 = regex_1.search(inference_text)
        match_2 = regex_2.search(inference_text)
        
        # If match_1 is found, extract and process the JSON data using pattern_1
        if match_1:
            # Extract the matched data as a dictionary
            json_data = match_1.groupdict()
            
            # Add Image information
            json_data['Image'] = item['Image']
            
            return json_data
        # If match_2 is found, extract and process the JSON data using pattern_2
        elif match_2:
            # Extract the JSON string
            json_string = match_2.group(1)
            
            # Replace single quotes with double quotes for property names
            json_string = json_string.replace("'", '"')
            
            try:
                # Load JSON data
                json_data = json.loads(json_string)
                
                # Add Image information
                json_data['Image'] = item['Image']
                
                return json_data
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for image {item['Image']}: {e}")
                return None
        else:
            # Neither pattern matched, return None
            return None
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Process items in parallel and retrieve the results
        json_outputs = list(filter(None, executor.map(process_item, final_results)))
    
    return json_outputs

In [10]:
def extract_image_base(image):
    # Extract the base part of the image name, excluding any part numbers
    match = re.match(r'(.+)_part\d+\.jpg$', image)
    if match:
        return match.group(1)
    return image

def convert_to_int_if_possible(value):
    try:
        # Convert to float first, then to int if no decimal part remains
        float_value = float(value)
        if float_value.is_integer():
            return int(float_value)
        return float_value
    except ValueError:
        return value

def process_fields(events):
    for event in events:
        for key, value in event.items():
            # Apply conversion only to fields that can be numeric
            if isinstance(value, str):
                # Strip trailing decimals from numeric strings
                event[key] = convert_to_int_if_possible(value)
            elif isinstance(value, (int, float)):
                # Convert numeric types directly
                event[key] = convert_to_int_if_possible(value)

def group_events_by_image(events):
    # Extract base identifiers for sorting and grouping
    for event in events:
        event['Image_Base'] = extract_image_base(event['Image'])
    
    # Process fields to remove extra decimal places
    process_fields(events)
    
    # Sort events by the 'Image_Base' to ensure groups are contiguous
    events.sort(key=itemgetter('Image_Base'))
    
    # Group events by 'Image_Base'
    grouped_events = [list(group) for key, group in groupby(events, key=itemgetter('Image_Base'))]
    
    # Flatten the list of groups into a single list, removing 'Image_Base'
    sorted_events = []
    for group in grouped_events:
        for event in group:
            del event['Image_Base']
        sorted_events.extend(group)
    
    return sorted_events

In [11]:
def duplicate_keys(input_list):
    output_list = []

    for item in input_list:
        new_item = {}
        for key, value in item.items():
            if '_orig' in key and key != 'Image_orig':
                new_key = f"{key.replace('_orig', '')}_hwr"
                new_item[key] = value
                new_item[new_key] = value
            else:
                new_item[key] = value
                
        output_list.append(new_item)

    return output_list

In [12]:
def get_confidence_and_prect_direct(key_value, confidence_data, bounding_box_data):
    try:
        words = key_value.split() if isinstance(key_value, str) else key_value.get('text', '').split()
        confidences = []
        bounding_boxes = []  # This will store bounding boxes for individual words

        for word in words:
            # Find and append the confidence score
            for confidence_word, confidence in confidence_data:
                if word.lower() == confidence_word.lower():  # Use case-insensitive comparison
                    confidences.append(confidence)
                    break

            # Find and append the bounding box
            for bbox_word, bbox in bounding_box_data:
                if word.lower() == bbox_word.lower():
                    # Assume bbox is in the format of (top_left_x, top_left_y, bottom_right_x, bottom_right_y)
                    # and that it belongs to the correct part of the image.
                    bounding_boxes.append((bbox[0][0], bbox[0][1], bbox[2][0], bbox[2][1]))
                    break  # Assuming the first match is used, similar to the confidence lookup

        # If there are no bounding boxes, return default values
        if not bounding_boxes:
            return '0', 0, 0, 0, 0  # Return default values for confidence and bounding box

        # Calculate average confidence
        average_confidence = str((sum(confidences) / len(confidences))) if confidences else '0'

        # Use the bounding box of the first word as the representative, if multiple words
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = bounding_boxes[0]

        return average_confidence, top_left_x, top_left_y, bottom_right_x, bottom_right_y
    except Exception as e:
        print(f"An error occurred in get_confidence_and_prect_direct: {str(e)}")
        return '0', 0, 0, 0, 0



def process_json_outputs(json_outputs, ocr_data, folder_name):
    try:
        processed_data = []
        batch_name = os.path.basename(folder_name)
        for json_output in json_outputs:
            modified_output = {
                'ImageType': 'Marriage',
                'Batch': batch_name,
                'Image': json_output.get('Image', ''),
                'ZoneUserID': '',
                'ZoneDate': '',
                'ImageRotation': '',
                'Line_Number': '',
                'State_File_orig': '',
                'State_File_hwr': '',
                'State_File_conf': '',
                'State_File_PRect': ''
            }
            image_name = json_output.get('Image', '')
            corresponding_ocr_result = next((item for item in ocr_data if item.get('Image', '') == image_name), None)
            confidence_data = corresponding_ocr_result.get('Confidence Scores', []) if corresponding_ocr_result else []
            bounding_box_data = corresponding_ocr_result.get('Bounding Boxes', []) if corresponding_ocr_result else []

            # Separate keys with '_orig' or '_hwr' from other keys
            orig_hwr_keys = [key for key in json_output.keys() if key.endswith('_orig') or key.endswith('_hwr')]
            other_keys = [key for key in json_output.keys() if key not in orig_hwr_keys]

            # Process keys with '_orig' or '_hwr' first
            for key in orig_hwr_keys:
                base_key = key.rsplit('_', 1)[0]  # Remove suffix like '_hwr' or '_orig'
                value = json_output[key]
                modified_output[key] = value
                if corresponding_ocr_result:
                    try:
                        confidence, x1_min, y1_min, x2_max, y2_max = get_confidence_and_prect_direct(value, confidence_data, bounding_box_data)
                        modified_output[f'{base_key}_conf'] = confidence
                        modified_output[f'{base_key}_PRect'] = str(x1_min) + ',' + str(y1_min) + ',' + str(x2_max) + ',' + str(y2_max)
                    except Exception as e:
                        print(f"An error occurred while processing key '{key}' for image '{image_name}': {str(e)}")

            # Process other keys
            for key in other_keys:
                base_key = key.rsplit('_', 1)[0]  # Remove suffix like '_hwr' or '_orig'
                value = json_output[key]
                modified_output[key] = value
                if corresponding_ocr_result and (key.endswith('_orig') or key.endswith('_hwr')):
                    try:
                        confidence, x1_min, y1_min, x2_max, y2_max = get_confidence_and_prect_direct(value, confidence_data, bounding_box_data)
                        modified_output[f'{base_key}_conf'] = confidence
                        modified_output[f'{base_key}_PRect'] = str(x1_min) + ',' + str(y1_min) + ',' + str(x2_max) + ',' + str(y2_max)
                    except Exception as e:
                        print(f"An error occurred while processing key '{key}' for image '{image_name}': {str(e)}")

            processed_data.append(modified_output)
        return processed_data
    except Exception as e:
        print(f"An error occurred in process_json_outputs: {str(e)}")
        return []

In [13]:
def clean_data(data, split_info):
    # Create a mapping of images to their coordinates
    split_info_dict = {info['Image']: info['coordinates'] for info in split_info}
    
    for entry in data:
        # Get the image name from the entry
        image_name = entry.get('Image')
        
        # Find the coordinates if the image name is in split_info_dict
        if image_name in split_info_dict:
            coordinates = split_info_dict[image_name]
            coordinates_str = ','.join(map(str, coordinates))
        
            # Iterate through keys and clean _PRect and _conf fields
            for key in list(entry.keys()):
                if key.endswith('_PRect'):
                    if entry[key] == '0,0,0,0':
                        entry[key] = coordinates_str
                elif key.endswith('_conf'):
                    if entry[key] == 0:
                        entry[key] = ' '
    
    return data

In [14]:
def organize_data(input_data):
    organized_data = []

    for entry in input_data:
        new_entry = {}
        for key, value in entry.items():
            # Skip 'Image_conf' and 'Image_PRect'
            if key in ['Image_conf', 'Image_PRect']:
                continue

            if key.endswith('_orig'):
                new_key_hwr = key.replace('_orig', '_hwr')
                new_entry[key] = value
                new_entry[new_key_hwr] = entry.get(new_key_hwr, '')
            elif key.endswith('_hwr'):
                continue
            else:
                new_entry[key] = value

        organized_data.append(new_entry)

    return organized_data

In [15]:
def update_image_names(dict_list):
    for item in dict_list:
        if 'Image' in item:
            # Split the image name by underscore and take the first two parts
            parts = item['Image'].split('_')
            if len(parts) > 1:
                # Combine the first two parts to get the new image name
                new_image_name = f"{parts[0]}_{parts[1]}"
                item['Image'] = new_image_name
    return dict_list

In [16]:
def write_to_pipe_delimited_file(list_of_dicts, filename=None):
    if not list_of_dicts:
        return ""
    headers = list(list_of_dicts[0].keys())
    header_line = '|'.join(headers)
    data_lines = []
    for entry in list_of_dicts:
        values = [str(entry.get(key, '')) for key in headers]
        data_lines.append('|'.join(values))
    result = f"{header_line}\n"
    result += '\n'.join(data_lines)
    if filename:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(result)

In [17]:
start_time = time.time()
# input_folder = folder_to_process_path
# file_name = output_txt_path + ".txt"
input_folder="/root/NA0219/NA0219_Test Images"
file_name="NA0219_test_images.txt"
results = process_image_folder(input_folder, max_workers=8)
image_with_ocr = get_image_ocr_list(ocrs)
outputs = process_ocr_data(image_with_ocr, ft_model, tokenizer, batch_size=10)
final_results = merge_lists(outputs, ocrs)
llm_json_output = extract_json_outputs_parallel(final_results, max_workers=8)
orig_duplicate = duplicate_keys(llm_json_output)
p = split_bbox_info(bounding_boxes_info)
filtered_ocr = extract_filtered_info_v2(ocrs, split_info)
processed_data = process_json_outputs(orig_duplicate, filtered_ocr, input_folder)
cleaned_data = clean_data(processed_data,split_info)
sorted_data = group_events_by_image(cleaned_data)
organized_data = organize_data(sorted_data)
updated_list = update_image_names(organized_data)
write_to_pipe_delimited_file(updated_list, file_name)
end_time = time.time()
elapsed_time = end_time - start_time
print("\nExecution Completed!!!!!")
print(f"It took {elapsed_time} seconds to execute")


0: 448x640 4 recordss, 79.4ms
Speed: 8.9ms preprocess, 79.4ms inference, 775.7ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 4 recordss, 76.4ms
Speed: 3.4ms preprocess, 76.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.7ms
Speed: 2.9ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.5ms
Speed: 2.6ms preprocess, 9.5ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.4ms
Speed: 2.5ms preprocess, 9.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.6ms
Speed: 2.6ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.6ms
Speed: 2.5ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 recordss, 9.7ms
Speed: 2.7ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1,

Processing Images: 100%|██████████| 30/30 [01:58<00:00,  3.96s/it]


Total OCR entries: 120


Processing batches:   0%|          | 0/12 [00:00<?, ?batch/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Processing batch from index 0 to 10


...


Processing batch from index 100 to 110


Processing batches:  92%|█████████▏| 11/12 [1:02:22<05:41, 341.10s/batch]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



Processing batch from index 110 to 120


Processing batches: 100%|██████████| 12/12 [1:07:53<00:00, 339.44s/batch]



Execution Completed!!!!!
It took 4195.998757839203 seconds to execute


In [18]:
# import matplotlib.pyplot as plt
# import os
# import cv2
# model = YOLO("/root/NA0219/last_5-8-24.pt")
# def detect_records(input_folder):
#     # Iterate over all images in the input folder
#     for filename in os.listdir(input_folder):
#         if filename.endswith(('.jpg', '.jpeg', '.png')):
#             img_path = os.path.join(input_folder, filename)
#             img = cv2.imread(img_path)
#             results = model(img)
#             for result in results:
#                 if result.boxes.data.shape[0] > 0:  # Check for detections
#                     for i, box in enumerate(result.boxes.data.tolist()):
#                         xmin, ymin, xmax, ymax, conf, cls = box

#                         # Draw the bounding box on the image
#                         cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 5)

#             plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             plt.title(f"Detections on {filename}")
#             plt.axis('off')
#             plt.show()

# input_folder = "/root/NA0219/NA0219_test_images"
# detect_records(input_folder)

In [23]:
len(llm_json_output)/len(final_results)

1.0